# MMDA Prohibited roads for e-bike

The MMDA Regulation No. 24-022 prohibited roads for e-bike are as follows:

1. **C1:** Recto Avenue
2. **C2:** Pres. Quirino Avenue
3. **C3:** Araneta Avenue
4. **C4:** EDSA
5. **C5:** Katipunan/CP Garcia
6. **C6:** Southeast Metro Manila Expressway
7. **R1:** Roxas Boulevard
8. **R2:** Taft Avenue
9. **R3:** Osmeña Highway
10. **R4:** Shaw Boulevard
11. **R5:** Ortigas Avenue
12. **R6:** Magsaysay Blvd./Aurora Blvd.
13. **R7:** Quezon Ave./Commonwealth Ave.
14. **R8:** A. Bonifacio Ave.
15. **R9:** Rizal Ave.
16. **R10:** Del Pan/Marcos Highway/McArthur Highway
17. Elliptical Road
18. Mindanao Avenue
19. Marcos Highway
20. Boni Avenue (Mandaluyong)
21. España Blvd.

We'll have to map out these roads/edges into a set of edges `prohibited_roads` so that we can remove them from our main graph, and also embed the `prohibited_roads` edges in our map (to show that our Dijkstra routes avoid them)

## Issues

The roads/edges in OpenStreetmaps are cut-off into multiple parts. A stretch of road (say Boni Avenue) corresponds to multiple edges in our graph which can each have a different name (or have a list of names instead of a single one, or worse have no name at all)

- For each road in the MMDA prohibited roads, inspect the main graph edges using the `.explore` function to check what names were used in the OSM graph for that road (e.g. C4: EDSA have different names in our graph: )



In [54]:
import osmnx as ox
import networkx as nx
import pandas as pd
weight = "length"

In [55]:
# Load Metro Manila graph
G = ox.io.load_graphml(filepath="data/metro_drive.graphml")
nodes, edges = ox.graph_to_gdfs(G)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x0000020F5F1A8BD0>>
Traceback (most recent call last):
  File "C:\Users\NITRO 5\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [ ]:
# This maps out all the edges in our graph. Useful for manual insepction of roads but is very laggy
#edges.explore(tiles="cartodbdarkmatter", style_kwds={"weight": 3})

In [ ]:
#  Highway is an attribute of OSM roads/edges, find all which are either motorway, trunk, or primary since they are the most important highway tags/categories
cf = '["highway"~"motorway|trunk|primary"]' # https://wiki.openstreetmap.org/wiki/Key:highway
primary_G = ox.graph_from_place("Metro Manila, Philippines", network_type="drive", custom_filter=cf)
primary_nodes, primary_edges = ox.graph_to_gdfs(G)

## Filter for highway types: motorway, trunk, primary is a shortcut to quickly get most of the prohibited roads

Since the MMDA prohibited roads for e-bikes are national, circumferential, and radial roads (roads which could be deemed as the most *important* roads) we can easily get most of them by filtering the OpenStreetMap graph edges for those that have highway type motorway, trunk, or primary (which are the most *important* highway tags in OpenStreetMaps)

Hence, we can start creating our `prohibited_roads` by filling it with all the roads/edge with highway type motorway/trunk/primary.

However, there would still be some roads/edges we could have missed: either their highway tags did not match motorway/trunk/primary or had no highway tags altogether. We have to add them to our `prohibited_roads`

Also, some roads has the highway tag motorway/trunk/primary but are not part of the MMDA prohibited roads. We have to remove them from our `prohibited_roads`

In [ ]:
#primary_edges.explore(tiles="cartodbdarkmatter", style_kwds={"weight": 2, "opacity": 0.2})

Most of the MMDA prohibited roads are already captured by filtering for highway type motorway/trunk/primary. The first one I noticed not part of this set of motorway/trunk/primary road edges is Boni Avenue. For the rest of the MMDA prohibited roads, inspect if it is in this set of primary edges (search online for some reference on where the stretch of the road is)

Already checked the following are part of the `primary_edges`:

- Elliptical Road
- Marcos Highway
- Espana Boulevard
- Mindanao Avenue

For the rest of the Circumferential (C1-C6) and Radial (R1-R10) roads, refer to this map to get a rouch sketch https://www.google.com/maps/d/viewer?mid=1XjNUjlg6KEnrl9OkZ_0PDKj5ay4&hl=en_US

Note that some Radial and Circumferential roads go beyond the land boundary of Metro Manila. We can just disregard them

In [ ]:
# Initialize our prohibited roads with this set of primary edges
prohibited_roads = primary_edges.copy(deep=True)
Motorway = prohibited_roads[prohibited_roads["highway"] == "motorway"]
#Motorway.explore(tiles="cartodbdarkmatter", style_kwds={"weight": 2, "opacity": 0.2})
Trunk = prohibited_roads[prohibited_roads["highway"] == "trunk"]
#Trunk.explore(tiles="cartodbdarkmatter", style_kwds={"weight": 2, "opacity": 0.2})
Primary = prohibited_roads[prohibited_roads["highway"] == "primary"]
#Primary.explore(tiles="cartodbdarkmatter", style_kwds={"weight": 2, "opacity": 0.2})

prohibited_roads = pd.concat([Motorway, Trunk, Primary])
#prohibited_roads.explore(tiles="cartodbdarkmatter", style_kwds={"weight": 2, "opacity": 0.2})


### Some roads/edges have null names

Just let them be for now

In [ ]:
null_edges = edges[pd.isnull(edges.name)]
#null_edges.explore(tiles="cartodbdarkmatter")

## Example road inspections

### Map out Boni Avenue

Boni Avenue is part of the MMDA prohibited roads but was not part of our initial filtering with highway types of motorway/trunk/primary (since from inspection it has the `secondary` type) so we have to manually identify the roads/edges forming Boni Avenue

In [ ]:
boni_edges = edges[edges.name == "Boni Avenue"]
#boni_edges.explore(tiles="cartodbdarkmatter")

In [ ]:
C1_name= edges[edges['name'].isin(["C. M. Recto Avenue", "P. Casal Street", "Ayala Boulevard","Finance Road"])]
C1_osmid = edges[edges['osmid'].isin([704746733,704746733,1035321076,54030024,156307489,156307490,89542274,22739770,16176832,704741995,1065577643,704741994,22739769, 1072039865 ])]
C1_edges = pd.concat([C1_name, C1_osmid])

#C1_edges.explore(tiles="cartodbdarkmatter")

In [ ]:
C2_name= edges[edges['name'].isin(["Capulong Street", "Tayuman Street", "Lacson Avenue", "Mabini Flyover","Quirino Avenue","Mabini Bridge", "President Quirino Avenue"])]
C2_remove_osmid = C2_name[~C2_name['osmid'].isin([16355830,1072003272,9136639 ])]
#C2_osmid = edges[edges['osmid'].isin([15302562, 22694403, 22694405, 921182982,1601922353, 22694404, 4491076, 921182983, 9867481, 758153177, 706105473, 707164828, 7061103876,904921145 ])]
Mabini_bridge = edges[edges['length'].isin([594.454, 564.007])]
C2_final = C2_remove_osmid[~C2_remove_osmid['highway'].isin(["unclassified",])]

C2_edges = pd.concat([C2_final, Mabini_bridge])
#C2_edges.explore(tiles="cartodbdarkmatter")

#issue: included yung quirino avenue sa baba tho prohibited din sha since included sha sa R2

In [ ]:
C3_name_N= edges[edges['name'].isin(["C-3 Road", "Sergeant Rivera Avenue", "Gregorio Araneta Avenue"])]
C3_North = C3_name_N

C3_South_N = edges[edges['name'].isin(["South Avenue", "Ayala Avenue", "Ayala Avenue Extension", "McKinley Road", "McKinley Parkway"])]
C3_South_OSMID = edges[edges['osmid'].isin([223089274])]
C3_South = pd.concat([C3_South_N, C3_South_OSMID])
C3_South_exclusion = C3_South[~C3_South['osmid'].isin([1066535166])]

C3_edges = pd.concat([C3_North, C3_South_exclusion])
#C3_edges.explore(tiles="cartodbdarkmatter")


In [ ]:
C4_name= edges[edges['name'].isin(["C-4 Road", "EDSA", "Epifanio de los Santos Avenue", "EDSA Extension", "EDSA Flyover", "EDSA Carousel", "EDSA Ramp", "EDSA Service Road", "EDSA Northbound", "EDSA Southbound", "EDSA Northbound Ramp", "EDSA Southbound Ramp", "EDSA Northbound Service Road", "EDSA Southbound Service Road", "EDSA Northbound Flyover", "EDSA Southbound Flyover", "EDSA Northbound Carousel", "EDSA Southbound Carousel","General San Miguel Street", "Samson Road"])]

C4_edges = C4_name[~C4_name['highway'].isin(["residential",])]
#C4_edges.explore(tiles="cartodbdarkmatter")

In [ ]:
C5_name = edges[edges['name'].isin(["NLEX-Mindanao Avenue Link", "Mindanao Avenue", "Congressional Avenue", "Tandang Sora Avenue", "Katipunan Avenue", "E. Rodriguez Jr. Avenue","Colonel Bonny Serrano Avenue", "Carlos P. Garcia Avenue","C-5 Ortigas Flyover","C-5 Road Extension","C-5 Dagohoy Service Road"])]
C5_OSMID = edges[edges['osmid'].isin([226045895,[61893649, 1211247461], 574402290, [62047289,26137123,23489197,238972167],[38802629,27672590,27672591],574402290,1035328756,574402289,[23395225,1067217033,23395315,4308749],[238981576,5036105,23395317]])] 

C5_edges = pd.concat([C5_name, C5_OSMID])
#C5_edges.explore(tiles="cartodbdarkmatter")

In [ ]:
C6_name = edges[edges['name'].isin(["Laguna Lake Highway"])]
C6_edges = C6_name
#C6_edges.explore(tiles="cartodbdarkmatter")

In [ ]:
R4_name = edges[edges['name'].isin(["Jose P. Rizal Avenue Extension", "Jose P. Rizal Avenue", "Tejeron Street","Pedro Gil Street"])]
R4_edges = R4_name[~R4_name['osmid'].isin([9867880,130576792,[54422837,1068072813, 698290031],[526957115, 154422837], 900617858,[1068072813, 154422837, 698290031],[154422837,1068072813, 698290031, 963839126]])]
#R4_edges.explore(tiles="cartodbdarkmatter")

### Quezon City: East Avenue, North Avenue, West Avenue and Timog Avenue

Our initial filtering with highway types of motorway/trunk/primary in `primary_edges` contains these 4 Quezon City avenues, but they are not part of the prohibited roads. We have to remove them from our set of `prohibited_roads`

In [ ]:
# Filtering using isin
qc_edges = edges[edges['name'].isin(["East Avenue", "Timog Avenue", "North Avenue", "West Avenue"])]
#qc_edges.explore( tiles="cartodbdarkmatter")

## Stitching it all together

Hence from our `prohibited_roads` which currently holds all the `primary_edges` which just contains all roads with highway type motorway/trunk/primary, we have to:

- Add `boni_edges`
- Remove `qc_edges`
- For the rest of the C1-C6 roads and R1-R10 roads, check if it was successfully capture
    - **TODO**: Add all missing edges (if there are missing edges)
    - **TODO**: Remove all edges that are not actually prohibited (if there are edges wrongfully added from `primary_edges`)

Not sure what the pandas functions are to add/remove edges to `prohibited_roads`

In [ ]:
# Create the prohibited roads and final bawaldaanan
Clean = prohibited_roads[~prohibited_roads['name'].isin(["East Avenue", "Timog Avenue", "North Avenue", "West Avenue"])]
Completion = edges[edges['name'].isin(["Mel Lopez Boulevard", "F. B. Harrison Street", "Legarda Street", "Belfast Avenue", "A. Mendoza Street", "Laon Laan Street"])]
BawalNaDaan = pd.concat([Clean, C1_edges, C2_edges, C3_edges, C4_edges, C5_edges, C6_edges, boni_edges, Completion, R4_edges])

# Remove extra osmids and names
Deletion = BawalNaDaan[~BawalNaDaan['osmid'].apply(lambda x: isinstance(x, list) and any(item in x for item in [154644770,34905006, 1067403858, 136052903, 1262660448, 644424829, 645337996, 1109616685, 1109616686, 644424833, 653015383, 645165801, 636737586, 592027551, 24158899, 592216401, 9473123, 610046351, 709075246, 1245171754, 872810594, 1210656446, 55207610, 1123113151, 5019444, 1210656447, 431355691, 579555574, 579555575, 46990157, 963839126, 963839125, 963839125, 963839126, 860898318, 923581224, 923581225, 46990752, 46990830, 24054188, 734355081, 734355080, 645337994, 632555213]) or x in [34905006, 1067403858, 136052903, 1262660448, 592027551, 24158899, 592216401, 9473123, 610046351, 709075246, 1245171754, 872810594, 1210656446, 55207610, 1123113151, 5019444, 1210656447, 46990157, 963839126, 963839125, 860898318, 923581224, 923581225, 46990752, 46990830, 24054188, 734355081, 734355080, 645337994, 632555213,154644770,15834431])]
Final_Bawaldaanan = Deletion[~Deletion['name'].isin(["General Paulino Santos Avenue", "Daang Hari", "Investment Drive","Scout Memorial Circle"])]


#Final_Bawaldaanan.explore(tiles="cartodbdarkmatter", style_kwds={"weight": 2, "opacity": 0.5})

#Checked Roads (R1, R2, R3, R4, R5, R6, R7, R8, R9, R10 , C1-C6)

## Create new Metro Manila edges for the prohibited roads, and the Metro Manila edges without the prohibited roads

In [61]:

# Metro edges with prohibited roads
metro_edges_prohibited_roads = edges[edges.index.isin(Final_Bawaldaanan.index)]
#metro_edges_prohibited_roads.explore(tiles="cartodbdarkmatter", style_kwds={"weight": 2, "opacity": 0.5})

# Metro Manila edges without the prohibited roads 
metro_edges_without_prohibited = edges[~edges.index.isin(Final_Bawaldaanan.index)]
#metro_edges_without_prohibited.explore(tiles="cartodbdarkmatter", style_kwds={"weight": 2, "opacity": 0.5})



## Output with OSMnx

Now that we have all the edges of Metro Manila without the prohibited roads, we need to create an new graph using the original `nodes` and the new edges `metro_edges_without_prohibited`. Then save it to disk


In [62]:
#Export graph of Metro Manila without the prohibited roads
metro_graph_without_prohibited = ox.graph_from_gdfs(nodes, metro_edges_without_prohibited)

# Export graph of primary nodes and primary edges for the prohibited roads (since we're going to map the prohibited roads)
metro_graph_prohibited = ox.graph_from_gdfs(primary_nodes, metro_edges_prohibited_roads)

In [63]:
# Save the graph to disk
ox.io.save_graphml(metro_graph_without_prohibited, filepath="data/metro_graph_without_prohibited.graphml")
ox.io.save_graphml(metro_graph_prohibited, filepath="data/metro_graph_prohibited.graphml")